# Disaggregate Shapefile

Disaggregates a shapefile column over a raster grid using the `intersection_map.p` file created by `create_shapefile_raster_intersection_file_parallel.py`.

In [2]:
import rasterio
import fiona
import rtree
from shapely.geometry import shape, Polygon

import cPickle as pickle

from collections import defaultdict

In [ ]:
input_fn = "boundary_shapefiles/counties/tl_2010_us_county00_trimmed_epsg4326.shp"

In [ ]:
f = rasterio.open("lloyd_hires_usa_data/mastergrid_countryarea_1km.tif", "r")
output_shape = f.shape
output_meta = f.meta.copy()
nodata_mask = f.read() != 9471966 # grab all pixels that aren't part of the US
f.close()

In [2]:
intersection_map = pickle.load(open("derived_data/county00_intersection_map.p", "rb"))

In [8]:
tract_cell_map = defaultdict(list)

for y in xrange(output_shape[0]):
    if y % 100 == 0:
        print("%d/%d" % (y, output_shape[0]))
    for x in range(output_shape[1]):
        
        for k,v in intersection_map[y][x].items():            
            tract_cell_map[k].append(((x,y), v))

0/2984
100/2984
200/2984
300/2984
400/2984
500/2984
600/2984
700/2984
800/2984
900/2984
1000/2984
1100/2984
1200/2984
1300/2984
1400/2984
1500/2984
1600/2984
1700/2984
1800/2984
1900/2984
2000/2984
2100/2984
2200/2984
2300/2984
2400/2984
2500/2984
2600/2984
2700/2984
2800/2984
2900/2984


In [37]:
data = np.zeros(output_shape, dtype=float)

f = fiona.open(input_fn, "r")
count = 0
total_pop = 0.0
for entry in f:
    
    if count % 1000 == 0:
        print(count)
    
    geoid = int(entry["properties"]["CNTYIDFP00"])
    population = entry["properties"]["SF1_P00100"]
    total_pop += population
        
    if len(tract_cell_map[geoid]) > 0:
    
        coverage = 0.0
        for (x,y), percentage in tract_cell_map[geoid]:
            coverage += percentage

        unit_population = float(population) / coverage

        for (x,y), percentage in tract_cell_map[geoid]:
            data[y,x] += percentage * unit_population
    else:
        # no intersection cells
        #if population > 0:
        #    print("Unallocated population, geoid=%s, population=%d" % (geoid, population))
    
        pass
    
    count += 1
    
f.close()

0
1000
2000
3000


In [39]:
total_pop

279583437.0

In [40]:
data.sum()

279583436.99999952

In [41]:
total_pop - data.sum()

4.76837158203125e-07

In [42]:
data[nodata_mask] = -1

output_meta["dtype"] = "float64"
output_meta["nodata"] = -1

f = rasterio.open("test.tif", "w", **output_meta)
f.write(data, 1)
f.close()